# 도서 유사도 분석 : Word2Vec & Cosine Similarity

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf
from pyspark.sql import HiveContext
from pyspark.sql import functions as F
import numpy as np
from numpy import dot
from numpy.linalg import norm
sc = pyspark.SparkContext.getOrCreate()
sc.version

'2.4.8'

In [3]:
sc = SparkSession.builder.appName('book').getOrCreate()

## 문화빅데이터플랫폼 도서별 키워드 데이터

In [6]:
keyword_data ='hdfs://master:9000/home/hadoop/project1/book_keyword.csv'
keyword_df = sc.read.csv(keyword_data, header=True, inferSchema=True)

In [21]:
keyword_df.limit(60).toPandas()

,_c0,isbn13,term
0,0,9780140372564,div
1,1,9780140372564,놀이기구
2,2,9780375825262,주디스 그린버그
3,3,9780375825262,액션 어드벤처
4,4,9780439099103,미국
5,5,9780439099103,위주
6,6,9780439099103,수학
7,7,9780439099103,초등학생구성
8,8,9780439099103,언어
9,9,9780439099103,과학


## isbn으로 그룹화하여 도서별 전체 키워드를 한 배열에 저장

In [6]:
keyword_list_df = keyword_df.groupby("isbn13").agg(F.collect_list("term").alias('term_list'))
keyword_list_df.limit(4).toPandas()

,isbn13,term_list
0,8809264181287,"[동물, 아기, 촉감, 놀이, 동시, 발달, 사운드, 소리, 두뇌, 하마, 앵무새,..."
1,8809639453452,"[div, 라스트크리스마스, 부탁하나만들어, 내여자친구의결혼식, 감독, 크레이지리치..."
2,9780061846854,"[강아지, 시리즈, 멋쟁이, 그림, 제인 오코너, HarperCollins, 그린,..."
3,9780399590597,"[워터댄서, 세상과나사이, 애틀랜틱, 미국, 매체, 타임, 수상작가, 대학, 베스트..."


## 크롤링 책 정보 데이터

In [7]:
book_data = "hdfs://master:9000/home/hadoop/project1/book_info.csv"
book_df = sc.read.option('multiline', True).csv(book_data, header=True, inferSchema=True)

In [8]:
book_df.limit(10).toPandas()

,book_id,main_category,sub_category,book_title,book_author,publisher,publish_date,aladin_price,aladin_star,aladin_review,...,yes24_star,yes24_review,interpark_price,interpark_star,interpark_review,min price,avg stars,num of reviews,book_isbn,book_trailer
0,182678,어린이,초등1~2학년,종이 봉지 공주,버트 먼치,비룡소,2008.07.18,9900,9.2,131,...,9.0,76,9900,9.2,56,9900,9.250,264,9788949110479,왕자를 구한 공주의 운명은?\r\n\r\n안데르센의 동화 「돼지치기 소년」을 페미니...
1,2084345,역사/문화,역사/문화일반,총 균 쇠,재레드 다이아몬드,문학사상사,2005.12.19,25200,9.0,301,...,9.2,241,25200,9.2,134,25200,9.125,677,9788970127248,왜 어떤 민족들은 다른 민족들의 정복과 지배의 대상으로 전락하고 말았는가. 왜 원주...
2,6050510,어린이,초등5~6학년,불량한 자전거 여행,남중,창비,2009.07.28,9720,9.6,92,...,9.5,79,9720,9.8,22,9720,9.600,196,9788936442507,"뜨거운 여름 1,100킬로 자전거 여행 한 소년이 그 길에서 준비한 '불량한 출발'..."
3,6055863,소설,고전/문학,1Q84,무라카미 하루키,문학동네,2009.09.08,13320,8.6,327,...,8.7,130,13320,9.2,203,13320,9.050,692,9788954608657,당신의 하늘에는 몇 개의 달이 떠 있습니까?\r\n\r\n무라카미 하루키가 <어둠의...
4,6348922,인문,종교학,종교의 미래,하비 콕스,문예출판사,2010.08.25,15300,9.1,11,...,8.4,14,15300,8.9,10,15300,8.800,42,9788931006810,세계적인 종교학자 하비콕스가 내놓은 그리스도교의 미래와 전망\r\n\r\n예수의 시...
5,6597874,취미/레저,옷만들기,초보자를 위한 친환경 가구 만들기,우상연,북하우스엔,2011.03.19,18000,8.2,20,...,8.6,9,18000,9.3,10,18000,8.600,42,9788956055152,친절한 가구 제작 안내서\r\n\r\n공구 사용법부터 가구 제작법까지 우리집 목공 ...
6,6793587,해외도서,취미/여행,Pantone Postcard Box: 100 Postcards (Novelty),Pantone INC,Chronicle Books (CA),2011.06.22,16500,0.0,0,...,9.2,12,17500,0.0,0,16500,4.175,13,9780811877541,WITH A PALETTE DRAWN FROM THE SYSTEMS OF PANTO...
7,6822225,청소년,청소년 문학,방관자,제임스 프렐러,미래인,2012.03.05,8550,9.2,29,...,8.8,43,8550,9.4,27,8550,9.300,100,9788983946966,방관자는 다음 피해자인가?\r\n\r\n가해자와 피해자 사이에서 갈등하는 방관자의 ...
8,7820011,예술/대중문화,사진,가문비나무의 노래,마틴 슐레스케,니케북스,2014.07.10,13500,10.0,5,...,9.3,17,13500,9.4,6,13500,9.650,28,9788994361147,독일의 바이올린 장인 마틴 슐레스케가 작업장에서 길어 올린 365개의 맑은 생각과 ...
9,8882146,역사/문화,역사/문화일반,역사란 무엇인가,에드워드 카,까치,2015.03.16,10800,9.2,68,...,8.7,35,10800,8.8,2,10800,9.050,106,9788972915812,역사란 ‘과거와 현재의 대화’ 또는 ‘과거의 사실과 현재의 역사가의 대화’라는 것은...


## 불필요한 컬럼 제거

In [9]:
book_df = book_df.drop('main_category', 'publish_date', 'aladin_price', 'aladin_star', 'aladin_review',
        'kyobo_price', 'kyobo_star', 'kyobo_review', 'yes24_price', 'yes24_star',
        'yes24_review', 'interpark_price', 'interpark_star', 'interpark_review', 'min price',
        'avg stars', 'num of reviews', 'publisher', 'book_trailer','book_author')

In [10]:
book_df.limit(10).toPandas()

,book_id,sub_category,book_title,book_isbn
0,182678,초등1~2학년,종이 봉지 공주,9788949110479
1,2084345,역사/문화일반,총 균 쇠,9788970127248
2,6050510,초등5~6학년,불량한 자전거 여행,9788936442507
3,6055863,고전/문학,1Q84,9788954608657
4,6348922,종교학,종교의 미래,9788931006810
5,6597874,옷만들기,초보자를 위한 친환경 가구 만들기,9788956055152
6,6793587,취미/여행,Pantone Postcard Box: 100 Postcards (Novelty),9780811877541
7,6822225,청소년 문학,방관자,9788983946966
8,7820011,사진,가문비나무의 노래,9788994361147
9,8882146,역사/문화일반,역사란 무엇인가,9788972915812


## 도서별 키워드 데이터와 크롤링한 도서 정보 데이터를 ISBN을 기준으로 Join

In [11]:
book_keyword_df = book_df.join(keyword_list_df, book_df.book_isbn == keyword_list_df.isbn13).select(book_df.book_id, book_df.book_isbn, book_df.book_title, book_df.sub_category, keyword_list_df.term_list)


In [12]:
book_keyword_df.limit(10).toPandas()

,book_id,book_isbn,book_title,sub_category,term_list
0,15556614,8809264181287,하마를 간질간질하지 마세요!,유아그림책,"[동물, 아기, 촉감, 놀이, 동시, 발달, 사운드, 소리, 두뇌, 하마, 앵무새,..."
1,7716485,9788936464325,가지 않은 길,외국시,"[휘트먼, 앨런, 월트, 미국시의, 시인, 15인의, 흐름, 포우, 미국, 로버트,..."
2,16363117,9788958206682,나무가 좋아지는 나무책,백과/전문사전,"[나무, 나무가좋아지는나무책, 광릉숲이야기, 해설가, 생강나무, 사람, 시작, 국립..."
3,17583515,9788974798789,조용헌의 영지순례,기타,"[기운, 영지, 지리산, 도사, 조용헌, 가면, 마음, 에너지, 오대산, 달래, 풍..."
4,17524835,9791157062157,거의 모든 IT의 역사,마케팅/세일즈,"[역사, 애플, 혁명, 거의모든IT의역사, 구글, 마이크로소프트, 컴퓨터, 세상, ..."
5,21026070,9791158741297,공식의 아름다움,과학기본서,"[공식, 방정식, 인류, 정리, 응용편, 수학, 문명, 이론편, 양자학파, 이론, ..."
6,18990051,9791164137879,흔한남매 별난 방탈출,어린이(공통),"[탈출, 흔한남매, 흔한남매별난방탈출2, 방탈출시리즈, 코믹, 미션, 콘텐츠, 테스..."
7,16122320,9791190427616,0~5세 뇌가 쑥쑥 자라는 놀이 육아,임신/출산/육아,"[놀이, 아기, 발달, 거울, 자극, 아이, 이보연, 부모, 손가락, 세상, 궁금,..."
8,20791207,9791190938822,땅의 역사,여행 에세이,"[역사, 하늘, 대처, 인물, 간신, 자세, 백성, 군상, 부부, 연산군, 사람, ..."
9,6286293,9788974578312,미국교과서 읽는 리딩 Easy 3,어린이영어,"[미국교과서읽는리딩, 영어, 키출판사, 미국, 미국교과서, 원어민, 교과서, 학습,..."


## Spark ML Word2Vec을 이용하여 도서 정보 임베딩

In [13]:
from pyspark.ml.feature import Word2Vec

word2vec = Word2Vec(vectorSize = 100, minCount = 2, inputCol = 'term_list', outputCol = 'vector')
word2vec.setMaxIter(20)
model = word2vec.fit(book_keyword_df)
book_w2v = model.transform(book_keyword_df)

In [14]:
# result : average word vector for each data
book_w2v.limit(10).toPandas()

,book_id,book_isbn,book_title,sub_category,term_list,vector
0,15556614,8809264181287,하마를 간질간질하지 마세요!,유아그림책,"[동물, 아기, 촉감, 놀이, 동시, 발달, 사운드, 소리, 두뇌, 하마, 앵무새,...","[-0.11413172258471604, -0.07202943800948561, 0..."
1,7716485,9788936464325,가지 않은 길,외국시,"[휘트먼, 앨런, 월트, 미국시의, 시인, 15인의, 흐름, 포우, 미국, 로버트,...","[0.08269175697118043, -0.015240778549470835, 0..."
2,16363117,9788958206682,나무가 좋아지는 나무책,백과/전문사전,"[나무, 나무가좋아지는나무책, 광릉숲이야기, 해설가, 생강나무, 사람, 시작, 국립...","[0.06030477740245664, 0.13132103735956155, -0...."
3,17583515,9788974798789,조용헌의 영지순례,기타,"[기운, 영지, 지리산, 도사, 조용헌, 가면, 마음, 에너지, 오대산, 달래, 풍...","[0.08150884236640005, 0.011724913237380114, -0..."
4,17524835,9791157062157,거의 모든 IT의 역사,마케팅/세일즈,"[역사, 애플, 혁명, 거의모든IT의역사, 구글, 마이크로소프트, 컴퓨터, 세상, ...","[-0.017973519541888, 0.03285391755249819, 0.00..."
5,21026070,9791158741297,공식의 아름다움,과학기본서,"[공식, 방정식, 인류, 정리, 응용편, 수학, 문명, 이론편, 양자학파, 이론, ...","[-0.055173547533972155, 0.01812285324427073, -..."
6,18990051,9791164137879,흔한남매 별난 방탈출,어린이(공통),"[탈출, 흔한남매, 흔한남매별난방탈출2, 방탈출시리즈, 코믹, 미션, 콘텐츠, 테스...","[0.06282540109302652, 0.003016470468872668, 0...."
7,16122320,9791190427616,0~5세 뇌가 쑥쑥 자라는 놀이 육아,임신/출산/육아,"[놀이, 아기, 발달, 거울, 자극, 아이, 이보연, 부모, 손가락, 세상, 궁금,...","[0.05154163262437383, -0.0551401888272169, 0.0..."
8,20791207,9791190938822,땅의 역사,여행 에세이,"[역사, 하늘, 대처, 인물, 간신, 자세, 백성, 군상, 부부, 연산군, 사람, ...","[0.08028392798695813, 0.15007073137057114, -0...."
9,6286293,9788974578312,미국교과서 읽는 리딩 Easy 3,어린이영어,"[미국교과서읽는리딩, 영어, 키출판사, 미국, 미국교과서, 원어민, 교과서, 학습,...","[-0.021350186440395193, -0.0449956194479455, 0..."


## Cosine Similarity

In [51]:
# sparkd에서 cosine similarity를 제공하지 않아서 numpy를 이용하여 계산

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

## 서브카테고리 데이터

In [18]:
category_data = 'hdfs://master:9000/home/hadoop/project1/book_category.csv'
category_df = sc.read.csv(category_data, header=True, inferSchema=True)

In [28]:
category_df = category_df.drop("main_category", "url")

## Cosine Similarity를 이용한 Vector간 유사도 분석

In [59]:
result = list()
for row in category_df.collect():
    sub_slice_df = book_w2v.filter("sub_category = '"+row.sub_category+"'")
    for book_row in sub_slice_df.collect():
        for book_row_target in sub_slice_df.collect():
            if(book_row.book_id == book_row_target.book_id):
                continue
            else:
                result.append([book_row.book_id, book_row_target.book_id, cos_sim(book_row.vector, book_row_target.vector)])   
    print(row.sub_category + " 서브카테고리 분석 완료")
    

나라별 소설 서브카테고리 분석 완료
고전/문학 서브카테고리 분석 완료
장르소설 서브카테고리 분석 완료
테마소설 서브카테고리 분석 완료
한국시 서브카테고리 분석 완료
외국시 서브카테고리 분석 완료
그림/포토 에세이 서브카테고리 분석 완료
독서 에세이 서브카테고리 분석 완료
명상 에세이 서브카테고리 분석 완료
성공 에세이 서브카테고리 분석 완료
여행 에세이 서브카테고리 분석 완료
연애/사랑 에세이 서브카테고리 분석 완료
인물 에세이 서브카테고리 분석 완료
삶의 지혜/명언 서브카테고리 분석 완료
경제 서브카테고리 분석 완료
경영 서브카테고리 분석 완료
마케팅/세일즈 서브카테고리 분석 완료
재테크/투자 서브카테고리 분석 완료
창업/취업 서브카테고리 분석 완료
대화/협상 서브카테고리 분석 완료
성공/처세 서브카테고리 분석 완료
시간관리 서브카테고리 분석 완료
인간관계 서브카테고리 분석 완료
자기능력계발 서브카테고리 분석 완료
인문일반 서브카테고리 분석 완료
심리 서브카테고리 분석 완료
교육학 서브카테고리 분석 완료
철학 서브카테고리 분석 완료
문학론 서브카테고리 분석 완료
언어학/기호학 서브카테고리 분석 완료
종교학 서브카테고리 분석 완료
신화 서브카테고리 분석 완료
역사/문화일반 서브카테고리 분석 완료
세계사 서브카테고리 분석 완료
서양사 서브카테고리 분석 완료
동양사 서브카테고리 분석 완료
한국사 서브카테고리 분석 완료
정치/외교 서브카테고리 분석 완료
행정/정책 서브카테고리 분석 완료
국방/군사/경찰 서브카테고리 분석 완료
법 서브카테고리 분석 완료
사회학 서브카테고리 분석 완료
사회복지 서브카테고리 분석 완료
언론/신문/방송 서브카테고리 분석 완료
공학일반/산업공학 서브카테고리 분석 완료
과학기본서 서브카테고리 분석 완료
기계/전기/전자 서브카테고리 분석 완료
농수산/축산 서브카테고리 분석 완료
도시/토목/건설 서브카테고리 분석 완료
물리학 서브카테고리 분석 완료
생물학 서브카테고리 분석 완료
수학 서브카테고리 분석 완료
쉽게 읽는 과학 서브카테고리 분석 완료


/home/hadoop/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


경영경제 서브카테고리 분석 완료
과학/공학/수학 서브카테고리 분석 완료
문학/소설 서브카테고리 분석 완료
사회/정치/법 서브카테고리 분석 완료
언어/외국어/사전 서브카테고리 분석 완료
요리 서브카테고리 분석 완료
유아청소년/교육 서브카테고리 분석 완료
종교/명상 서브카테고리 분석 완료
철학/심리/역사 서브카테고리 분석 완료
취미/여행 서브카테고리 분석 완료
컴퓨터 서브카테고리 분석 완료
독일 서브카테고리 분석 완료
스페인 서브카테고리 분석 완료
일본도서 서브카테고리 분석 완료
프랑스 서브카테고리 분석 완료


## 분석 결과 csv 저장

In [60]:
import pandas as pd

book_similarity_word2vec = pd.DataFrame(result)
book_similarity_word2vec.columns = ['book_id', 'target_book_id', 'score']
book_similarity_word2vec = SparkSession.builder.getOrCreate().createDataFrame(book_similarity_word2vec)                

In [62]:
book_similarity_word2vec = book_similarity_word2vec.orderBy(col('book_id').asc(), col('score').desc())

In [63]:
book_similarity_word2vec.limit(10).toPandas()

,book_id,target_book_id,score
0,132610,16376412,0.705072
1,132610,16824000,0.655311
2,132610,8778810,0.569145
3,132610,20492659,0.507864
4,132610,11490924,0.469026
5,185888,17470709,0.785674
6,185888,18896391,0.766852
7,185888,20870332,0.739423
8,185888,18904246,0.717394
9,185888,21141848,0.647083


In [64]:
book_similarity_word2vec.coalesce(1).write.format("com.databricks.spark.csv"). \
                            option("header", "true").save("book_similarity_word2vec.csv")

In [112]:
# 저장한 파일 읽기
result_data = 'word2vec_similarity.csv'
result_df = sc.read.csv(result_data, header=True, inferSchema=True)

In [113]:
result_df.limit(10).toPandas()

,book_id,target_book_id,score
0,132610,16376412,0.705072
1,132610,16824000,0.655311
2,132610,8778810,0.569145
3,132610,20492659,0.507864
4,132610,11490924,0.469026
5,185888,17470709,0.785674
6,185888,18896391,0.766852
7,185888,20870332,0.739423
8,185888,18904246,0.717394
9,185888,21141848,0.647083


## 분석 결과 확인

결과를 한눈에 확인하기 위해 book_title을 포함하도록 조인하였습니다.

In [95]:
word2vec_similarity_result = book_similarity_word2vec.join(book_df, book_similarity_word2vec.target_book_id == book_df.book_id).select(book_similarity_word2vec.book_id, book_similarity_word2vec.target_book_id, book_df.book_title, book_similarity_word2vec.score)

In [96]:
word2vec_similarity_result = word2vec_similarity_result.withColumnRenamed("book_title","target_book_title")

In [98]:
word2vec_similarity_result = word2vec_similarity_result.join(book_df, word2vec_similarity_result.book_id == book_df.book_id).select(word2vec_similarity_result.book_id, book_df.book_title, word2vec_similarity_result.target_book_id, word2vec_similarity_result.target_book_title, word2vec_similarity_result.score)

In [111]:
word2vec_similarity_result.limit(5).toPandas()

,book_id,book_title,target_book_id,target_book_title,score
0,132610,이집트 신화,16376412,최초의 신화 길가메쉬 서사시,0.705072
1,132610,이집트 신화,16824000,세계 괴물 백과,0.655311
2,132610,이집트 신화,8778810,구스타프 슈바브의 그리스로마 신화,0.569145
3,132610,이집트 신화,20492659,여성 귀신이 되다,0.507864
4,132610,이집트 신화,11490924,왜 그리스 신화를 읽어야 하나요?,0.469026
